In [19]:
# 布林带策略
# 数据：沪深300 日频数据 from JoinQuant
# 策略核心：
# 当收盘价'Close'上穿上轨线，买入
# 当收盘价'Close'下穿下轨线，卖空

# 定义上中下轨道 mid & up & down

In [20]:
import pandas as pd
import tushare as ts
import jqdatasdk
import os
import numpy as np

<font size=5>**沪深300日频数据读取**</font>

In [21]:
# 读取本地下载好的沪深300_daily csv文件并进行数据预处理
df_300 = pd.read_csv("000300_daily.csv", header=0, names=["DateTime", "Open", "Close", "High", "Low", "Volume", "Money"])
df_300.head()

,DateTime,Open,Close,High,Low,Volume,Money
0,2005-09-01,928.52,944.56,944.95,926.06,2.230638e+09,1.055868e+10
1,2005-09-02,945.98,947.87,947.92,941.31,1.861444e+09,8.864774e+09
2,2005-09-05,949.08,952.72,952.99,944.63,1.784332e+09,8.242226e+09
3,2005-09-06,953.41,936.61,956.30,934.97,2.122226e+09,1.058399e+10
4,2005-09-07,934.99,952.76,952.90,932.65,1.960847e+09,9.956808e+09


<font size=5>**计算基于Tp值的布林带上中下轨道**</font>

Tp值为研报定义的每日收盘价
上中下轨道计算均根据Tp值进行计算

In [22]:
# 算法 Mid = 最高价 + 最低价 + 收盘价 / 3 作为中轨
df_300['Tp'] =round((df_300['High'] + df_300['Low'] + df_300['Close']) / 3, 2) # Tp 当前价格 保留两位
df_300['Mid'] = np.nan # 中轨线=N日tp的移动平均线 均值拟合：N=20 20天移动平均
df_300['Up'] = np.nan  # 上轨线=中轨线+M倍的标准差(中轨线标准差) M=2 2倍标准差
df_300['Down'] = np.nan # 下轨线=中轨线-M倍的标准差（中轨线标准差）M=2 2倍标准差
df_300.head()  

,DateTime,Open,Close,High,Low,Volume,Money,Tp,Mid,Up,Down
0,2005-09-01,928.52,944.56,944.95,926.06,2.230638e+09,1.055868e+10,938.52,NaN,NaN,NaN
1,2005-09-02,945.98,947.87,947.92,941.31,1.861444e+09,8.864774e+09,945.70,NaN,NaN,NaN
2,2005-09-05,949.08,952.72,952.99,944.63,1.784332e+09,8.242226e+09,950.11,NaN,NaN,NaN
3,2005-09-06,953.41,936.61,956.30,934.97,2.122226e+09,1.058399e+10,942.63,NaN,NaN,NaN
4,2005-09-07,934.99,952.76,952.90,932.65,1.960847e+09,9.956808e+09,946.10,NaN,NaN,NaN


In [23]:
# 求中轨线Tp方法 1 for循环
'''构建中轨线Tp 方法1: for loop'''
# for i in range(19, len(df_300)):
'''想用赋值操作第一步先切出来用loc 再进行计算'''
#     df_300.loc[df_300.index[i], 'Mid'] = df_300['Tp'][i-19:i+1].mean()

# 求中轨线Tp方法 2 rolling(n) 窗口滑动求平均
# df_300['Mid'] = round(df_300['Tp'].rolling(20).mean(), 2)
# df_300['Up'] = round((df_300['Tp'].rolling(20).std()) * 2 + df_300['Tp'].rolling(20).mean(), 2)
# df_300['Down'] = round(df_300['Tp'].rolling(20).mean() - (df_300['Tp'].rolling(20).std()) * 2, 2)

# 写成函数形式
'''
该函数用于计算布林带的上中下轨道
Param: DataFrame; N; m
'''
def Construct_Band(df, N, m): # Dataframe, N = days of moving average, M = std倍数
    df['Mid'] = round(df['Tp'].rolling(N).mean(), 2)
    df['Up'] = round((df['Tp'].rolling(N).std()) * 2 + df['Mid'], 2)
    df['Down'] = round(df['Mid'] - (df['Tp'].rolling(N).std()) * 2, 2)
    return df


<font size=5>**20日均线 + 2倍标准差构建布林带**</font>

In [24]:
'''20日移动均线和两倍标准差构建BBand'''
Construct_Band(df_300, 20, 2)
df_300.head(40)

,DateTime,Open,Close,High,Low,Volume,Money,Tp,Mid,Up,Down
0,2005-09-01,928.52,944.56,944.95,926.06,2.230638e+09,1.055868e+10,938.52,NaN,NaN,NaN
1,2005-09-02,945.98,947.87,947.92,941.31,1.861444e+09,8.864774e+09,945.70,NaN,NaN,NaN
2,2005-09-05,949.08,952.72,952.99,944.63,1.784332e+09,8.242226e+09,950.11,NaN,NaN,NaN
3,2005-09-06,953.41,936.61,956.30,934.97,2.122226e+09,1.058399e+10,942.63,NaN,NaN,NaN
4,2005-09-07,934.99,952.76,952.90,932.65,1.960847e+09,9.956808e+09,946.10,NaN,NaN,NaN
5,2005-09-08,954.27,955.28,958.87,947.71,2.242439e+09,1.175326e+10,953.95,NaN,NaN,NaN
6,2005-09-09,955.11,949.07,959.90,946.02,1.621931e+09,7.924194e+09,951.66,NaN,NaN,NaN
7,2005-09-12,949.78,949.51,952.55,945.40,1.167261e+09,5.610130e+09,949.15,NaN,NaN,NaN
8,2005-09-13,949.58,963.77,963.92,948.13,1.657943e+09,7.999031e+09,958.61,NaN,NaN,NaN
9,2005-09-14,964.97,970.19,970.26,961.64,1.995829e+09,9.942628e+09,967.36,NaN,NaN,NaN


In [25]:
'''
策略回测
回测样本: 沪深300 - daily - 2005-09-01 至 2012-03-15
指标参数: N = 20 20日移动平均线构建布林带

策略信号
当收盘价Close 上穿上轨线Up, 买入, signal = 1      昨日收盘小于昨日上轨 & 今日收盘大于今日上轨
当收盘价Close 下穿下轨线Down, 卖出, signal = -1   昨日收盘大于昨日下轨 & 今日收盘小于今日下轨
'''
df_300


,DateTime,Open,Close,High,Low,Volume,Money,Tp,Mid,Up,Down
0,2005-09-01,928.52,944.56,944.95,926.06,2.230638e+09,1.055868e+10,938.52,NaN,NaN,NaN
1,2005-09-02,945.98,947.87,947.92,941.31,1.861444e+09,8.864774e+09,945.70,NaN,NaN,NaN
2,2005-09-05,949.08,952.72,952.99,944.63,1.784332e+09,8.242226e+09,950.11,NaN,NaN,NaN
3,2005-09-06,953.41,936.61,956.30,934.97,2.122226e+09,1.058399e+10,942.63,NaN,NaN,NaN
4,2005-09-07,934.99,952.76,952.90,932.65,1.960847e+09,9.956808e+09,946.10,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1583,2012-03-09,2644.10,2664.30,2664.41,2631.41,5.525892e+09,6.596344e+10,2653.37,2602.94,2710.08,2495.80
1584,2012-03-12,2663.24,2654.40,2666.43,2636.42,6.232722e+09,7.436770e+10,2652.42,2609.20,2712.26,2506.14
1585,2012-03-13,2653.68,2681.07,2681.33,2649.17,5.840447e+09,6.642976e+10,2670.52,2616.70,2714.30,2519.10
1586,2012-03-14,2694.47,2605.11,2705.75,2595.34,1.084265e+10,1.170804e+11,2635.40,2621.42,2712.52,2530.32


<font size=5>**构建策略函数并计算信号**</font>

In [26]:
'''Calculate signals 计算信号
计算出每天的信号是1 -1 or None
Param: df_300 DataFrame 
'''
def Signal_calculation(df):
    up = df['Up']        # List Container
    down = df['Down']    # List Container
    close = df['Close']  # List Container
    signals_list = []         # empty List Container
    for up_, down_, close_, pre_up, pre_down, pre_close in zip(up, down, close, up.shift(1), down.shift(1), close.shift(1)):
        signal_element = None
        '''昨日收盘小于昨日上轨 & 今日收盘大于今日上轨 signal = 1'''
        if (pre_close < pre_up) and (close_ >= up_):
            signal_element = 1
            '''昨日收盘大于昨日下轨 & 今日收盘小于今日下轨 signal = -1'''
        elif (pre_close >= pre_down) and (close_ < down_):
            signal_element = -1
        # 一次for循环添加一次  每次遍历signal重新变回none
        #  重新进行信号判断 要不添加none 要不添加1 -1在列表里 none也是一个元素添加进去            
        signals_list.append(signal_element) 
    '''把List signal赋值给dataFrame 作为新列表'''           
    df['Signal'] = signals_list
    return df
    


In [27]:
'''信号赋值 -> df_300['Signal]'''
Signal_calculation(df_300) # NaN是一个numpy.float64的非空对象

,DateTime,Open,Close,High,Low,Volume,Money,Tp,Mid,Up,Down,Signal
0,2005-09-01,928.52,944.56,944.95,926.06,2.230638e+09,1.055868e+10,938.52,NaN,NaN,NaN,NaN
1,2005-09-02,945.98,947.87,947.92,941.31,1.861444e+09,8.864774e+09,945.70,NaN,NaN,NaN,NaN
2,2005-09-05,949.08,952.72,952.99,944.63,1.784332e+09,8.242226e+09,950.11,NaN,NaN,NaN,NaN
3,2005-09-06,953.41,936.61,956.30,934.97,2.122226e+09,1.058399e+10,942.63,NaN,NaN,NaN,NaN
4,2005-09-07,934.99,952.76,952.90,932.65,1.960847e+09,9.956808e+09,946.10,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1583,2012-03-09,2644.10,2664.30,2664.41,2631.41,5.525892e+09,6.596344e+10,2653.37,2602.94,2710.08,2495.80,NaN
1584,2012-03-12,2663.24,2654.40,2666.43,2636.42,6.232722e+09,7.436770e+10,2652.42,2609.20,2712.26,2506.14,NaN
1585,2012-03-13,2653.68,2681.07,2681.33,2649.17,5.840447e+09,6.642976e+10,2670.52,2616.70,2714.30,2519.10,NaN
1586,2012-03-14,2694.47,2605.11,2705.75,2595.34,1.084265e+10,1.170804e+11,2635.40,2621.42,2712.52,2530.32,NaN


In [28]:
df_300['Signal']

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
        ..
1583   NaN
1584   NaN
1585   NaN
1586   NaN
1587   NaN
Name: Signal, Length: 1588, dtype: float64

In [29]:
'''check信号处理完后 check 列Signal中 信号为 NaN 所在的位置 发现有1476个NaN 总共有1588个数据 产生了 1588 - 1476 = 112个信号'''
# 输出 Signal列 的所有 NaN
# 为什么加转置？ 得到的每一行求any()计算的结果，输出为行的Series
df_300[df_300[['Signal']].isnull().T.any()][['Signal']]

,Signal
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
1583,NaN
1584,NaN
1585,NaN
1586,NaN


In [30]:
# NaN -> not a number
# NULL-> 不存在的东西，是空的
# a = np.isnan(df_300['Signal']) # Series 返回Boolean
df_300
df_300.to_csv('test300', index=False)

<font size=5>**根据信号计算持仓并调整DataFrame**</font>

In [31]:
'''计算持仓'''
# Position通过信号signal计算 当出现第一个信号1或者-1的时候, 信号之间的NaN自动抄录最上面碰到的第一个非NaN信号 
# 再把开头未出现信号的NaN
df_300['Position'] = df_300[['Signal']].fillna(method = 'ffill').fillna(0)
df_300 # Position 计算完毕 0 / -1 / 1

,DateTime,Open,Close,High,Low,Volume,Money,Tp,Mid,Up,Down,Signal,Position
0,2005-09-01,928.52,944.56,944.95,926.06,2.230638e+09,1.055868e+10,938.52,NaN,NaN,NaN,NaN,0.0
1,2005-09-02,945.98,947.87,947.92,941.31,1.861444e+09,8.864774e+09,945.70,NaN,NaN,NaN,NaN,0.0
2,2005-09-05,949.08,952.72,952.99,944.63,1.784332e+09,8.242226e+09,950.11,NaN,NaN,NaN,NaN,0.0
3,2005-09-06,953.41,936.61,956.30,934.97,2.122226e+09,1.058399e+10,942.63,NaN,NaN,NaN,NaN,0.0
4,2005-09-07,934.99,952.76,952.90,932.65,1.960847e+09,9.956808e+09,946.10,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1583,2012-03-09,2644.10,2664.30,2664.41,2631.41,5.525892e+09,6.596344e+10,2653.37,2602.94,2710.08,2495.80,NaN,1.0
1584,2012-03-12,2663.24,2654.40,2666.43,2636.42,6.232722e+09,7.436770e+10,2652.42,2609.20,2712.26,2506.14,NaN,1.0
1585,2012-03-13,2653.68,2681.07,2681.33,2649.17,5.840447e+09,6.642976e+10,2670.52,2616.70,2714.30,2519.10,NaN,1.0
1586,2012-03-14,2694.47,2605.11,2705.75,2595.34,1.084265e+10,1.170804e+11,2635.40,2621.42,2712.52,2530.32,NaN,1.0


In [32]:
# 计算回测指标以及结果
# 继续对沪深300_DataFrame进行预处理 计算出每日收盘价Close差价, 以及每日变化百分比
df_300['Pre_Close'] = df_300['Close'].shift(1)
df_300['Pre_Close'].fillna(950, inplace=True)
df_300['Change'] = df_300['Close'] - df_300['Pre_Close']
df_300['Pct_Change'] = round((df_300['Change'] / df_300['Pre_Close']) * 100,2)
df_300.to_csv('Preprocessing_Done.csv', index=False)
# 此时 DataFrame df_300全部处理完毕 'Position' 有全部的信号 'Signal'有NaN Pre_Close & Change & Pct_Change有全部的具体数值 

In [33]:
# 查看最终可用于指标计算的data frame
df_300

,DateTime,Open,Close,High,Low,Volume,Money,Tp,Mid,Up,Down,Signal,Position,Pre_Close,Change,Pct_Change
0,2005-09-01,928.52,944.56,944.95,926.06,2.230638e+09,1.055868e+10,938.52,NaN,NaN,NaN,NaN,0.0,950.00,-5.44,-0.57
1,2005-09-02,945.98,947.87,947.92,941.31,1.861444e+09,8.864774e+09,945.70,NaN,NaN,NaN,NaN,0.0,944.56,3.31,0.35
2,2005-09-05,949.08,952.72,952.99,944.63,1.784332e+09,8.242226e+09,950.11,NaN,NaN,NaN,NaN,0.0,947.87,4.85,0.51
3,2005-09-06,953.41,936.61,956.30,934.97,2.122226e+09,1.058399e+10,942.63,NaN,NaN,NaN,NaN,0.0,952.72,-16.11,-1.69
4,2005-09-07,934.99,952.76,952.90,932.65,1.960847e+09,9.956808e+09,946.10,NaN,NaN,NaN,NaN,0.0,936.61,16.15,1.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1583,2012-03-09,2644.10,2664.30,2664.41,2631.41,5.525892e+09,6.596344e+10,2653.37,2602.94,2710.08,2495.80,NaN,1.0,2635.79,28.51,1.08
1584,2012-03-12,2663.24,2654.40,2666.43,2636.42,6.232722e+09,7.436770e+10,2652.42,2609.20,2712.26,2506.14,NaN,1.0,2664.30,-9.90,-0.37
1585,2012-03-13,2653.68,2681.07,2681.33,2649.17,5.840447e+09,6.642976e+10,2670.52,2616.70,2714.30,2519.10,NaN,1.0,2654.40,26.67,1.00
1586,2012-03-14,2694.47,2605.11,2705.75,2595.34,1.084265e+10,1.170804e+11,2635.40,2621.42,2712.52,2530.32,NaN,1.0,2681.07,-75.96,-2.83


<font size=5>**计算指标以及回测结果**</font>

In [34]:
def statistic_performance(df_300, r0 = 0.023, data_period=1588): # Data period is 1588 rows
    position = df_300['Position']
    
    #     holding_Return :            持仓收益
    #     W_L_forHolding :            持仓胜负
    #     cum_HoldingReturn :         累计持仓收益
    #     drawback :                  回撤
    #     excess_HoldingReturn :      超额收益
     
    holding_Return = df_300['Pct_Change']/100 * position         # 持仓收益
    W_L_forHolding = holding_Return > 0                          # 返回boolean 收益率大于0即为Win
    cum_HoldingReturn = (1+holding_Return).cumprod() - 1
    drawback = (cum_HoldingReturn.cummax()-cum_HoldingReturn)/(1+cum_HoldingReturn).cummax()    
    excess_HoldingReturn= holding_Return-r0 / (2520*1440/data_period) # Data period is 1588 rows

    df_300['holding_Return'] = holding_Return
    df_300['W_L_forHolding'] = W_L_forHolding
    df_300['cum_HoldingReturn'] = cum_HoldingReturn
    df_300['drawback'] = drawback
    df_300['excess_HoldingReturn'] = excess_HoldingReturn
    

    '''多仓指标'''
    #     num_LongPos_open：                多仓开仓次数
    #     num_LongPos_win：                 多仓开仓盈利次数
    #     num_LongHolding_period：          多仓持有周期数
    #     num_LongHolding_Windperiod：      多仓持有盈利周期数
    
    '''空仓指标'''
    #     num_ShortPos_open：               空仓开仓次数
    #     num_ShortPos_win：                空仓开仓盈利次数
    #     num_ShortHolding_period：         空仓持有周期数
    #     num_ShortHolding_Winperiod：      空仓持有盈利周期数
    
    '''持仓指标'''
    #     num_HoldingPeriod：               持仓周期数（最后一笔未平仓订单也算）
    #     num_HoldingPeriod_win：           持仓盈利周期数（最后一笔未平仓订单也算）

    '''回撤 & 年化收益 & 标准差 & Shape Ratio'''
    #     Cum_Return：                      累计持仓收益
    #     max_Drawback：                    最大回撤
    #     annual_STD：                      年化标准差
    #     annual_Return：                   年化收益
    #     sharpRatio：                      夏普

    Cum_Return = cum_HoldingReturn.tolist()[-1]

    num_LongPos_open= 0 
    num_LongPos_win = 0
    num_LongHolding_period = 0
    num_LongHolding_Windperiod = 0
    num_ShortPos_open= 0 
    num_ShortPos_win = 0
    num_ShortHolding_period = 0
    num_ShortHolding_Winperiod = 0
    for w, r, pre_pos, pos in zip(W_L_forHolding, holding_Return, position.shift(1), position):
        # 有换仓（先结算上一次持仓，再初始化本次持仓）
        if pre_pos!=pos: 
            # 判断pre_pos非空：若为空则是循环的第一次，此时无需结算，直接初始化持仓即可
            if pre_pos == pre_pos:
                # 结算上一次持仓
                if pre_pos>0:
                    num_LongPos_open += 1
                    num_LongHolding_period += tmp_hold_period
                    num_LongHolding_Windperiod += tmp_holdWinPeriod
                    if tmp_holdingReturn>0:
                        num_LongPos_win+=1
                elif pre_pos<0:
                    num_ShortPos_open += 1      
                    num_ShortHolding_period += tmp_hold_period
                    num_ShortHolding_Winperiod += tmp_holdWinPeriod
                    if tmp_holdingReturn>0:                    
                        num_ShortPos_win+=1
            # 初始化本次持仓
            tmp_holdingReturn = r
            tmp_hold_period = 0
            tmp_holdWinPeriod = 0
        else: # 未换仓
            if abs(pos)>0:
                tmp_hold_period += 1
                if r>0:
                    tmp_holdWinPeriod += 1
                if abs(r)>0:
                    tmp_holdingReturn = (1+tmp_holdingReturn)*(1+r)-1       

    num_HoldingPeriod = (abs(position)>0).sum()
    num_HoldingPeriod_win = (holding_Return>0).sum()
    max_Drawback = drawback.max()    
    annual_Return = pow( 1+Cum_Return, 
                      1/(data_period/1440*len(df_300)/250) )-1
    annual_STD = excess_HoldingReturn.std() * np.sqrt(250*1440/data_period) 
    sharpRatio= annual_Return / annual_STD

    """Performance DataFrame"""
    performance_cols = ['累计收益', 
                        '多仓次数', '多仓胜率', '多仓平均持有期', 
                        '空仓次数', '空仓胜率', '空仓平均持有期', 
                        '日胜率', '最大回撤', '年化收益/最大回撤',
                        '年化收益', '年化标准差', '年化夏普'
                       ]
    performance_values = ['{:.2%}'.format(Cum_Return),
                          num_LongPos_open, '{:.2%}'.format(num_LongPos_win/num_LongPos_open), 
                                            '{:.2f}'.format(num_LongHolding_period/num_LongPos_open),
                          num_ShortPos_open, '{:.2%}'.format(num_ShortPos_win/num_ShortPos_open), 
                                            '{:.2f}'.format(num_ShortHolding_period/num_ShortPos_open),
                          '{:.2%}'.format(num_HoldingPeriod_win/num_HoldingPeriod), 
                          '{:.2%}'.format(max_Drawback), 
                          '{:.2f}'.format(annual_Return/max_Drawback),
                          '{:.2%}'.format(annual_Return), 
                          '{:.2%}'.format(annual_STD), 
                          '{:.2f}'.format(sharpRatio)
                         ]
    performance_df = pd.DataFrame(performance_values, index=performance_cols)
    return df_300, performance_df

In [35]:
final_df_300, performance_df = statistic_performance(df_300)

In [36]:
final_df_300

,DateTime,Open,Close,High,Low,Volume,Money,Tp,Mid,Up,...,Signal,Position,Pre_Close,Change,Pct_Change,holding_Return,W_L_forHolding,cum_HoldingReturn,drawback,excess_HoldingReturn
0,2005-09-01,928.52,944.56,944.95,926.06,2.230638e+09,1.055868e+10,938.52,NaN,NaN,...,NaN,0.0,950.00,-5.44,-0.57,-0.0000,False,0.000000,0.000000,-0.00001
1,2005-09-02,945.98,947.87,947.92,941.31,1.861444e+09,8.864774e+09,945.70,NaN,NaN,...,NaN,0.0,944.56,3.31,0.35,0.0000,False,0.000000,0.000000,-0.00001
2,2005-09-05,949.08,952.72,952.99,944.63,1.784332e+09,8.242226e+09,950.11,NaN,NaN,...,NaN,0.0,947.87,4.85,0.51,0.0000,False,0.000000,0.000000,-0.00001
3,2005-09-06,953.41,936.61,956.30,934.97,2.122226e+09,1.058399e+10,942.63,NaN,NaN,...,NaN,0.0,952.72,-16.11,-1.69,-0.0000,False,0.000000,0.000000,-0.00001
4,2005-09-07,934.99,952.76,952.90,932.65,1.960847e+09,9.956808e+09,946.10,NaN,NaN,...,NaN,0.0,936.61,16.15,1.72,0.0000,False,0.000000,0.000000,-0.00001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1583,2012-03-09,2644.10,2664.30,2664.41,2631.41,5.525892e+09,6.596344e+10,2653.37,2602.94,2710.08,...,NaN,1.0,2635.79,28.51,1.08,0.0108,True,53.082754,0.005787,0.01079
1584,2012-03-12,2663.24,2654.40,2666.43,2636.42,6.232722e+09,7.436770e+10,2652.42,2609.20,2712.26,...,NaN,1.0,2664.30,-9.90,-0.37,-0.0037,False,52.882648,0.009466,-0.00371
1585,2012-03-13,2653.68,2681.07,2681.33,2649.17,5.840447e+09,6.642976e+10,2670.52,2616.70,2714.30,...,NaN,1.0,2654.40,26.67,1.00,0.0100,True,53.421474,0.000000,0.00999
1586,2012-03-14,2694.47,2605.11,2705.75,2595.34,1.084265e+10,1.170804e+11,2635.40,2621.42,2712.52,...,NaN,1.0,2681.07,-75.96,-2.83,-0.0283,False,51.881347,0.028300,-0.02831


In [37]:
performance_df

,0
累计收益,5148.47%
多仓次数,13
多仓胜率,76.92%
多仓平均持有期,58.62
空仓次数,14
空仓胜率,78.57%
空仓平均持有期,51.93
日胜率,57.08%
最大回撤,24.05%
年化收益/最大回撤,3.16
